In [1]:
import os

import matplotlib.pyplot as plt

import torchvision.datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
from util import download_mnist
from util import test, train, train_many_epochs
from SNN import SpikingNet

In [3]:
batch_size = 1000
DATA_PATH = './data'

training_set, testing_set = download_mnist(DATA_PATH)
train_set_loader = torch.utils.data.DataLoader(
    dataset=training_set,
    batch_size=batch_size,
    shuffle=True)

test_set_loader = torch.utils.data.DataLoader(
    dataset=testing_set,
    batch_size=batch_size,
    shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training SNN

In [4]:
spiking_model = SpikingNet(device, n_time_steps = 128, begin_eval=0)
train_many_epochs(spiking_model, device, train_set_loader, test_set_loader)

Train Epoch: 1 [0/60000 (0%)] Loss: 2.407422 Accuracy: 7.20%
Train Epoch: 1 [10000/60000 (17%)] Loss: 2.032497 Accuracy: 13.50%
Train Epoch: 1 [20000/60000 (33%)] Loss: 1.558197 Accuracy: 55.90%
Train Epoch: 1 [30000/60000 (50%)] Loss: 0.769982 Accuracy: 73.40%
Train Epoch: 1 [40000/60000 (67%)] Loss: 0.439049 Accuracy: 86.60%
Train Epoch: 1 [50000/60000 (83%)] Loss: 0.470174 Accuracy: 86.40%


/home/parvus/.local/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.0004, Accuracy: 8843/10000 (88.43%)

Train Epoch: 2 [0/60000 (0%)] Loss: 0.395777 Accuracy: 87.40%
Train Epoch: 2 [10000/60000 (17%)] Loss: 0.406506 Accuracy: 87.50%
Train Epoch: 2 [20000/60000 (33%)] Loss: 0.365729 Accuracy: 89.40%
Train Epoch: 2 [30000/60000 (50%)] Loss: 0.384646 Accuracy: 89.30%
Train Epoch: 2 [40000/60000 (67%)] Loss: 0.361450 Accuracy: 89.30%
Train Epoch: 2 [50000/60000 (83%)] Loss: 0.331238 Accuracy: 91.60%

Test set: Average loss: 0.0003, Accuracy: 9080/10000 (90.80%)

Train Epoch: 3 [0/60000 (0%)] Loss: 0.350239 Accuracy: 90.40%


KeyboardInterrupt: 